Задание

  1.Взять датасет
  https://huggingface.co/datasets/merionum/ru_paraphraser
  решить задачу парафраза



In [1]:
!pip install transformers sentencepiece datasets --quiet

     |████████████████████████████████| 4.7 MB 32.0 MB/s 
     |████████████████████████████████| 1.3 MB 65.2 MB/s 
     |████████████████████████████████| 365 kB 70.7 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
     |████████████████████████████████| 596 kB 72.0 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 212 kB 76.6 MB/s 
     |████████████████████████████████| 115 kB 62.0 MB/s 
     |████████████████████████████████| 141 kB 73.3 MB/s 
     |████████████████████████████████| 127 kB 70.6 MB/s 


In [9]:
import pandas as pd
from datasets import load_dataset

import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer

In [3]:
corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-57cfc8af008bb08a/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [7]:
print(corpus['train']['text_1'][7])

Васильева признана виновной в мошенничестве и хищениях


In [8]:
print(corpus['train']['text_2'][7])

Васильева признана виновной в хищениях и отмывании денег


In [10]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [13]:
result_df = []

def paraphrase_check(row_numb):
    
    text1 = corpus['train']['text_1'][row_numb]
    text2 = corpus['train']['text_2'][row_numb]
    
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).numpy()
    
    p = proba[0]
    
    result_df.append([text1, text2, p[1]])
    
    return pd.DataFrame(result_df, columns=['текст 1', 'текст 2', 'вероятность парафраза'])

In [14]:
paraphrase_check(0)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301


In [15]:
paraphrase_check(7)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796


In [16]:
paraphrase_check(14)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796
2,Следственный комитет сообщил о взломе своего с...,В СКР сообщили о взломе хакерами сайта ведомства,0.964652


In [17]:
paraphrase_check(20)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796
2,Следственный комитет сообщил о взломе своего с...,В СКР сообщили о взломе хакерами сайта ведомства,0.964652
3,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144


In [18]:
paraphrase_check(25)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796
2,Следственный комитет сообщил о взломе своего с...,В СКР сообщили о взломе хакерами сайта ведомства,0.964652
3,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144
4,9 мая метрополитен Петербурга будет работать к...,"Посол США – РБК: «Горд, что буду представлять ...",0.051736


In [19]:
paraphrase_check(30)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796
2,Следственный комитет сообщил о взломе своего с...,В СКР сообщили о взломе хакерами сайта ведомства,0.964652
3,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144
4,9 мая метрополитен Петербурга будет работать к...,"Посол США – РБК: «Горд, что буду представлять ...",0.051736
5,"Прокуратура пока не решила, обжаловать ли приг...",Защита Васильевой обжалует приговор суда,0.239415


In [20]:
paraphrase_check(50)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796
2,Следственный комитет сообщил о взломе своего с...,В СКР сообщили о взломе хакерами сайта ведомства,0.964652
3,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144
4,9 мая метрополитен Петербурга будет работать к...,"Посол США – РБК: «Горд, что буду представлять ...",0.051736
5,"Прокуратура пока не решила, обжаловать ли приг...",Защита Васильевой обжалует приговор суда,0.239415
6,Премьер Словакии прибыл в Москву для участия в...,Путеводитель по Дню Победы: как провести 9 мая...,0.053352


In [21]:
paraphrase_check(100)

,текст 1,текст 2,вероятность парафраза
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301
1,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796
2,Следственный комитет сообщил о взломе своего с...,В СКР сообщили о взломе хакерами сайта ведомства,0.964652
3,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144
4,9 мая метрополитен Петербурга будет работать к...,"Посол США – РБК: «Горд, что буду представлять ...",0.051736
5,"Прокуратура пока не решила, обжаловать ли приг...",Защита Васильевой обжалует приговор суда,0.239415
6,Премьер Словакии прибыл в Москву для участия в...,Путеводитель по Дню Победы: как провести 9 мая...,0.053352
7,Путин впервые объявил минуту молчания на парад...,Путин на параде Победы указал на опасности «бл...,0.053989


**Предобученная модель решает задачу парафраза достаточно хорошо.**